In [14]:
import numpy as np
import pandas as pd

# Pandas library for the pandas dataframes
import pandas as pd    
import numpy as np
import scipy.stats as stats
import scipy

# Import Scikit-Learn library for decision tree models
import sklearn         
from sklearn import linear_model, datasets
from sklearn.utils import resample
from sklearn import tree
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.ensemble import BaggingRegressor, BaggingClassifier, RandomForestRegressor, RandomForestClassifier, GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.model_selection import train_test_split, LeaveOneOut, KFold, cross_validate, RandomizedSearchCV
from sklearn.metrics import r2_score, mean_squared_error, accuracy_score
from sklearn import metrics
from sklearn.utils import resample
from sklearn.linear_model import Lasso
from pandas import DataFrame

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [18]:
df = pd.read_csv('../dataset/hao/diameter_only.csv')
df

,in_source,in_amount_mmol,p_source,p_amount_mmol,first_sol,first_sol_amount_ml,TOP,TOP_amount_mmol,acid,acid_amount_mmol,...,thiol,thiol_amount_mmol,zinc,zinc_amount_mmol,other,other_amount_mmol,total_volume_ml,temp_c,time_min,diameter_nm
0,indium myristate,0.10,tris(trimethylsilyl)phosphine,0.10,octadecene,8.0,None,0.000000,None,0.0,...,dodecanethiol,0.05,zinc stearate,0.1,None,0.000000,8.012,300,30.0,2.30
1,indium chloride,10.00,sodium phosphide,10.00,4-ethylpyridine,50.0,trioctylphosphine,44.841900,None,0.0,...,None,0.00,None,0.0,None,0.000000,70.000,130,2880.0,5.00
2,"indium tris(N,N?-diisopropylacetamidinato)",0.90,tris(trimethylsilyl)phosphine,0.15,mesitylene,5.0,None,0.000000,palmitic acid,0.3,...,None,0.00,None,0.0,None,0.000000,5.077,150,1260.0,2.00
3,"indium tris(N,N?-diisopropylacetamidinato)",0.90,tris(trimethylsilyl)phosphine,0.15,mesitylene,5.0,None,0.000000,palmitic acid,0.3,...,None,0.00,None,0.0,None,0.000000,5.077,230,1260.0,4.10
4,indium chloride,0.30,tris(dimethylamino)phosphine,0.08,None,0.0,None,0.000000,None,0.0,...,None,0.00,zinc chloride,2.2,water,1.200000,5.002,180,15.0,1.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,indium oxalate,2.33,tris(trimethylsilyl)phosphine,3.00,None,0.0,trioctylphosphine,24.552424,None,0.0,...,None,0.00,None,0.0,trioctylphosphine oxide,1.011111,11.962,290,4320.0,4.80
67,indium oxalate,2.33,tris(trimethylsilyl)phosphine,3.00,None,0.0,trioctylphosphine,24.552424,None,0.0,...,None,0.00,None,0.0,trioctylphosphine oxide,1.011111,11.962,300,4320.0,6.00
68,indium chloride,2.26,tris(trimethylsilyl)phosphine,2.00,None,0.0,trioctylphosphine,17.936760,None,0.0,...,None,0.00,None,0.0,trioctylphosphine oxide,0.830000,8.830,270,1140.0,5.00
69,indium chloride,0.10,tris(trimethylsilyl)phosphine,0.10,None,0.0,trioctylphosphine,11.210475,None,0.0,...,None,0.00,zinc oleate,2.0,None,0.000000,5.000,280,60.0,2.30


In [19]:
#Checks if there are any columns with no values
df.isna().sum()

in_source              0
in_amount_mmol         0
p_source               0
p_amount_mmol          0
first_sol              0
first_sol_amount_ml    0
TOP                    0
TOP_amount_mmol        0
acid                   0
acid_amount_mmol       0
amine                  0
amine_amount_mmol      0
thiol                  0
thiol_amount_mmol      0
zinc                   0
zinc_amount_mmol       0
other                  0
other_amount_mmol      0
total_volume_ml        0
temp_c                 0
time_min               0
diameter_nm            0
dtype: int64

In [30]:
#Separate out initial DataFrame into the input features and output features

df_input = df.drop(columns =['diameter_nm'], inplace = False, axis = 1) 
df_output = df['diameter_nm'].to_frame()


In [34]:
df_input.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71 entries, 0 to 70
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   in_source            71 non-null     object 
 1   in_amount_mmol       71 non-null     float64
 2   p_source             71 non-null     object 
 3   p_amount_mmol        71 non-null     float64
 4   first_sol            71 non-null     object 
 5   first_sol_amount_ml  71 non-null     float64
 6   TOP                  71 non-null     object 
 7   TOP_amount_mmol      71 non-null     float64
 8   acid                 71 non-null     object 
 9   acid_amount_mmol     71 non-null     float64
 10  amine                71 non-null     object 
 11  amine_amount_mmol    71 non-null     float64
 12  thiol                71 non-null     object 
 13  thiol_amount_mmol    71 non-null     float64
 14  zinc                 71 non-null     object 
 15  zinc_amount_mmol     71 non-null     float

In [35]:
#Converts the values in the "temp_c" column into float types
df_input['temp_c'] = df_input['temp_c'].astype(float)

df_input.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71 entries, 0 to 70
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   in_source            71 non-null     object 
 1   in_amount_mmol       71 non-null     float64
 2   p_source             71 non-null     object 
 3   p_amount_mmol        71 non-null     float64
 4   first_sol            71 non-null     object 
 5   first_sol_amount_ml  71 non-null     float64
 6   TOP                  71 non-null     object 
 7   TOP_amount_mmol      71 non-null     float64
 8   acid                 71 non-null     object 
 9   acid_amount_mmol     71 non-null     float64
 10  amine                71 non-null     object 
 11  amine_amount_mmol    71 non-null     float64
 12  thiol                71 non-null     object 
 13  thiol_amount_mmol    71 non-null     float64
 14  zinc                 71 non-null     object 
 15  zinc_amount_mmol     71 non-null     float

In [36]:
df_output.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71 entries, 0 to 70
Data columns (total 1 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   diameter_nm  71 non-null     float64
dtypes: float64(1)
memory usage: 696.0 bytes


In [38]:
#Initializes 2 lists to contain all of the numerical and categorical input columns
input_num_cols = [col for col in df_input.columns if df[col].dtypes !='O']
input_cat_cols = [col for col in df_input.columns if df[col].dtypes =='O']
input_cat_cols

['in_source',
 'p_source',
 'first_sol',
 'TOP',
 'acid',
 'amine',
 'thiol',
 'zinc',
 'other']

In [39]:
#Initializes the ColumnTransformer object, and specifies what it will do with a passed in dataframe
ct = ColumnTransformer([
    ('step1', StandardScaler(), input_num_cols),
    ('step2', OneHotEncoder(sparse=False, handle_unknown='ignore'), input_cat_cols)
], remainder = 'passthrough')

In [40]:
#Uses the ColumnTransformer object to modify the input columns
df_input_scaled_encoded = pd.DataFrame(ct.fit_transform(df_input))
df_input_scaled_encoded

,0,1,2,3,4,5,6,7,8,9,...,56,57,58,59,60,61,62,63,64,65
0,-0.670594,-0.880716,0.028488,-0.498168,-0.326407,-0.606822,8.366600,-0.506654,-0.419933,-0.176683,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,6.164180,5.816477,2.500872,4.453250,-0.326407,-0.606822,-0.119523,-0.614002,-0.419933,3.296294,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.118289,-0.846891,-0.148111,-0.498168,-0.223954,-0.560771,-0.119523,-0.614002,-0.419933,-0.341122,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-0.118289,-0.846891,-0.148111,-0.498168,-0.223954,-0.560771,-0.119523,-0.614002,-0.419933,-0.341122,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.532517,-0.894245,-0.442442,-0.498168,-0.326407,1.726461,-0.119523,1.747661,0.466107,-0.345324,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,0.868957,1.081088,-0.442442,2.212897,-0.326407,-0.606822,-0.119523,-0.614002,0.326638,0.044622,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
67,0.868957,1.081088,-0.442442,2.212897,-0.326407,-0.606822,-0.119523,-0.614002,0.326638,0.044622,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
68,0.820630,0.404604,-0.442442,1.482399,-0.326407,-0.606822,-0.119523,-0.614002,0.192911,-0.130854,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
69,-0.670594,-0.880716,-0.442442,0.739686,-0.326407,-0.606822,-0.119523,1.532965,-0.419933,-0.345436,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [41]:
#Now that we have transformed all of the relevant columns, how can we take back all of the relevant column names

ct.transformers_

[('step1',
  StandardScaler(),
  ['in_amount_mmol',
   'p_amount_mmol',
   'first_sol_amount_ml',
   'TOP_amount_mmol',
   'acid_amount_mmol',
   'amine_amount_mmol',
   'thiol_amount_mmol',
   'zinc_amount_mmol',
   'other_amount_mmol',
   'total_volume_ml',
   'temp_c',
   'time_min']),
 ('step2',
  OneHotEncoder(handle_unknown='ignore', sparse=False),
  ['in_source',
   'p_source',
   'first_sol',
   'TOP',
   'acid',
   'amine',
   'thiol',
   'zinc',
   'other'])]

In [42]:
#Look into ct object to see the column titles
array_of_cat_titles = ct.transformers_[1][1].get_feature_names()
len(array_of_cat_titles)

54

In [43]:
#Number of numerical columns in the dataset
len(input_num_cols)

12

In [44]:
for i in range(len(input_num_cols)):
    df_input_scaled_encoded.rename(columns={df_input_scaled_encoded.columns[i]: input_num_cols[i]}, inplace = True)

for j in range(len(array_of_cat_titles)):
    df_input_scaled_encoded.rename(columns={df_input_scaled_encoded.columns[i+1]: array_of_cat_titles[j]}, inplace = True)
    i = i + 1

In [46]:
df_input_scaled_encoded

,in_amount_mmol,p_amount_mmol,first_sol_amount_ml,TOP_amount_mmol,acid_amount_mmol,amine_amount_mmol,thiol_amount_mmol,zinc_amount_mmol,other_amount_mmol,total_volume_ml,...,x7_zinc stearate,x7_zinc undecylenate,x8_None,x8_copper bromide,x8_dioctyl ether,x8_oleic acid,x8_trioctylamine,x8_trioctylphosphine oxide,x8_water,x8_zinc iodide
0,-0.670594,-0.880716,0.028488,-0.498168,-0.326407,-0.606822,8.366600,-0.506654,-0.419933,-0.176683,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,6.164180,5.816477,2.500872,4.453250,-0.326407,-0.606822,-0.119523,-0.614002,-0.419933,3.296294,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.118289,-0.846891,-0.148111,-0.498168,-0.223954,-0.560771,-0.119523,-0.614002,-0.419933,-0.341122,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-0.118289,-0.846891,-0.148111,-0.498168,-0.223954,-0.560771,-0.119523,-0.614002,-0.419933,-0.341122,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.532517,-0.894245,-0.442442,-0.498168,-0.326407,1.726461,-0.119523,1.747661,0.466107,-0.345324,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,0.868957,1.081088,-0.442442,2.212897,-0.326407,-0.606822,-0.119523,-0.614002,0.326638,0.044622,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
67,0.868957,1.081088,-0.442442,2.212897,-0.326407,-0.606822,-0.119523,-0.614002,0.326638,0.044622,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
68,0.820630,0.404604,-0.442442,1.482399,-0.326407,-0.606822,-0.119523,-0.614002,0.192911,-0.130854,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
69,-0.670594,-0.880716,-0.442442,0.739686,-0.326407,-0.606822,-0.119523,1.532965,-0.419933,-0.345436,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [47]:
df_scaled_encoded = pd.concat([df_input_scaled_encoded, df_output], axis = 1)
df_scaled_encoded

,in_amount_mmol,p_amount_mmol,first_sol_amount_ml,TOP_amount_mmol,acid_amount_mmol,amine_amount_mmol,thiol_amount_mmol,zinc_amount_mmol,other_amount_mmol,total_volume_ml,...,x7_zinc undecylenate,x8_None,x8_copper bromide,x8_dioctyl ether,x8_oleic acid,x8_trioctylamine,x8_trioctylphosphine oxide,x8_water,x8_zinc iodide,diameter_nm
0,-0.670594,-0.880716,0.028488,-0.498168,-0.326407,-0.606822,8.366600,-0.506654,-0.419933,-0.176683,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.30
1,6.164180,5.816477,2.500872,4.453250,-0.326407,-0.606822,-0.119523,-0.614002,-0.419933,3.296294,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.00
2,-0.118289,-0.846891,-0.148111,-0.498168,-0.223954,-0.560771,-0.119523,-0.614002,-0.419933,-0.341122,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.00
3,-0.118289,-0.846891,-0.148111,-0.498168,-0.223954,-0.560771,-0.119523,-0.614002,-0.419933,-0.341122,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.10
4,-0.532517,-0.894245,-0.442442,-0.498168,-0.326407,1.726461,-0.119523,1.747661,0.466107,-0.345324,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,0.868957,1.081088,-0.442442,2.212897,-0.326407,-0.606822,-0.119523,-0.614002,0.326638,0.044622,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,4.80
67,0.868957,1.081088,-0.442442,2.212897,-0.326407,-0.606822,-0.119523,-0.614002,0.326638,0.044622,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,6.00
68,0.820630,0.404604,-0.442442,1.482399,-0.326407,-0.606822,-0.119523,-0.614002,0.192911,-0.130854,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,5.00
69,-0.670594,-0.880716,-0.442442,0.739686,-0.326407,-0.606822,-0.119523,1.532965,-0.419933,-0.345436,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.30


In [48]:
df_scaled_encoded.to_csv('InP_hao_diameter_only_scaled.csv')

In [49]:
df_scaled_encoded

,in_amount_mmol,p_amount_mmol,first_sol_amount_ml,TOP_amount_mmol,acid_amount_mmol,amine_amount_mmol,thiol_amount_mmol,zinc_amount_mmol,other_amount_mmol,total_volume_ml,...,x7_zinc undecylenate,x8_None,x8_copper bromide,x8_dioctyl ether,x8_oleic acid,x8_trioctylamine,x8_trioctylphosphine oxide,x8_water,x8_zinc iodide,diameter_nm
0,-0.670594,-0.880716,0.028488,-0.498168,-0.326407,-0.606822,8.366600,-0.506654,-0.419933,-0.176683,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.30
1,6.164180,5.816477,2.500872,4.453250,-0.326407,-0.606822,-0.119523,-0.614002,-0.419933,3.296294,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.00
2,-0.118289,-0.846891,-0.148111,-0.498168,-0.223954,-0.560771,-0.119523,-0.614002,-0.419933,-0.341122,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.00
3,-0.118289,-0.846891,-0.148111,-0.498168,-0.223954,-0.560771,-0.119523,-0.614002,-0.419933,-0.341122,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.10
4,-0.532517,-0.894245,-0.442442,-0.498168,-0.326407,1.726461,-0.119523,1.747661,0.466107,-0.345324,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,0.868957,1.081088,-0.442442,2.212897,-0.326407,-0.606822,-0.119523,-0.614002,0.326638,0.044622,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,4.80
67,0.868957,1.081088,-0.442442,2.212897,-0.326407,-0.606822,-0.119523,-0.614002,0.326638,0.044622,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,6.00
68,0.820630,0.404604,-0.442442,1.482399,-0.326407,-0.606822,-0.119523,-0.614002,0.192911,-0.130854,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,5.00
69,-0.670594,-0.880716,-0.442442,0.739686,-0.326407,-0.606822,-0.119523,1.532965,-0.419933,-0.345436,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.30
